### Pandas Lab: Time Shifts & Multi Level Indexing

This lab is designed to introduce you to working with time in a more granular way, and understanding how to build features when your data has hierarchies or panels.  

Ie, when you have repeated observations for the same objects.  This is an important concept because lots of statistical methods don't explicitly account for values which might naturally be correlated with one another over time.  

But lots of data **is** highly correlated over time!  

By the time you're done with this lab, you'll have built 9 columns that capture a variety of information about how an observed value is changing with respect to itself.

**Question 1:** Set the multi-level index so the first level is the Stock symbol itself, and the second level is the date.  Make sure the date column is sorted in ascending order.  You might have to use the `sort_index(level=0)` method to get the values straight.

In [116]:
import pandas as pd
df = pd.read_csv("/Users/aoifeduna/AoifeRepo/aoiferepo/Lectures/Unit2/data/stocks_panel.csv", parse_dates=['Date'])
# Don't forget to parse it as dates

In [117]:
df.head()

,Date,Stock,Price
0,2014-11-05,AAPL,108.860001
1,2014-11-05,AMZN,296.519989
2,2014-11-05,FB,74.830002
3,2014-11-05,MSFT,47.860001
4,2014-11-05,GOOGL,555.950012


In [118]:
df = df.set_index(['Stock', 'Date']).sort_index(level=0)

In [119]:
df.head()

Price
Stock Date                  
AAPL  2014-11-05  108.860001
      2014-11-06  108.699997
      2014-11-07  109.010002
      2014-11-10  108.830002
      2014-11-11  109.699997

**Question 2:** To capture some other aspects of dates, create columns in your dataset that capture this aspect of each timestamp:

  - What quarter it's in
  - Whether or not it's the last day of the month/quarter
  - What day it is (ie, do price changes vary by day?)
  
**Hint:** You don't use the `dt` attribute to get date parts from index values.  Multi indices are also a little tricky.  

To get what you want, try this: `df.index.get_level_values(level=1).your_datepart_here`

In [120]:
df.info()
# Checking to see if it's date time

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 6285 entries, (AAPL, 2014-11-05 00:00:00) to (MSFT, 2019-11-01 00:00:00)
Data columns (total 1 columns):
Price    6285 non-null float64
dtypes: float64(1)
memory usage: 77.5+ KB


In [121]:
df['Quarter'] = df.index.get_level_values(level=1).quarter
df.head()

Price  Quarter
Stock Date                           
AAPL  2014-11-05  108.860001        4
      2014-11-06  108.699997        4
      2014-11-07  109.010002        4
      2014-11-10  108.830002        4
      2014-11-11  109.699997        4

In [122]:
df['Day'] = df.index.get_level_values(level=1).dayofweek
df.head()

Price  Quarter  Day
Stock Date                                
AAPL  2014-11-05  108.860001        4    2
      2014-11-06  108.699997        4    3
      2014-11-07  109.010002        4    4
      2014-11-10  108.830002        4    0
      2014-11-11  109.699997        4    1

In [123]:
df['EndofMonth'] = df.index.get_level_values(level=1).is_month_end
df['EndofQuarter'] = df.index.get_level_values(level=1).is_quarter_end
df.head()

Price  Quarter  Day  EndofMonth  EndofQuarter
Stock Date                                                          
AAPL  2014-11-05  108.860001        4    2       False         False
      2014-11-06  108.699997        4    3       False         False
      2014-11-07  109.010002        4    4       False         False
      2014-11-10  108.830002        4    0       False         False
      2014-11-11  109.699997        4    1       False         False

**Question 3:** Time Series Embedding

Lots of times if you're trying to predict the value of something tomorrow, the most import piece of information is what the value of something is today, and yesterday, and so on.

Try and create columns that capture previously observed values for each stock.  

Make two columns that capture the value of the following:

 - What the previous recorded price for each stock was
 - The stock price from two observations ago
 
**Remember:** This has to be done on a particular level of the index to make sure it's getting applied appropriately!

In [124]:
df['PreviousValue'] = df.groupby(level=0)['Price'].shift()
df['Value2DaysAgo'] = df.groupby(level=0)['Price'].shift(2)
df.head(10)

Price  Quarter  Day  EndofMonth  EndofQuarter  \
Stock Date                                                             
AAPL  2014-11-05  108.860001        4    2       False         False   
      2014-11-06  108.699997        4    3       False         False   
      2014-11-07  109.010002        4    4       False         False   
      2014-11-10  108.830002        4    0       False         False   
      2014-11-11  109.699997        4    1       False         False   
      2014-11-12  111.250000        4    2       False         False   
      2014-11-13  112.820000        4    3       False         False   
      2014-11-14  114.180000        4    4       False         False   
      2014-11-17  113.989998        4    0       False         False   
      2014-11-18  115.470001        4    1       False         False   

                  PreviousValue  Value2DaysAgo  
Stock Date                                      
AAPL  2014-11-05            NaN            NaN  
      2014-11-06     108.860001            NaN  
      2014-11-07     108.699997     108.860001  
      2014-11-10     109.010002     108.699997  
      2014-11-11     108.830002     109.010002  
      2014-11-12     109.699997     108.830002  
      2014-11-13     111.250000     109.699997  
      2014-11-14     112.820000     111.250000  
      2014-11-17     114.180000     112.820000  
      2014-11-18     113.989998     114.180000

**Question 4:** How did each stock price change compared to the S&P 500? 

Lots of times it's useful to see how something else moves with some other item that you're trying to track.  

In the data folder is a file called `s&p.csv`, and it contains the price history of the S&P 500 index for each day since its inception. See if you can upload it, and merge the `adj close` column into your dataset, so there's a column that displays the observed value of the index for every single price observation we have in our dataset.

**Hints:**
 - Merging on multi-level indices is tricky and prone to failure.  To make this a little bit easier, just use `reset_index()` to pop out the date column in the multi-index, and merge on it as if it were a regular column.
 - Make sure both date columns are actually encoded as dates, rather than strings, or else the merge won't work.
 - You'll want to go back to the multi-level index when you're done with this step.

In [125]:
sp = pd.read_csv("/Users/aoifeduna/AoifeRepo/aoiferepo/Lectures/Unit2/data/s&p.csv", parse_dates=['Date'])
df = df.reset_index().merge(sp[['Date','Adj Close']], on='Date')

In [126]:
df = df.set_index(['Stock', 'Date']).sort_index(level=0)

In [127]:
df.head(10)

Price  Quarter  Day  EndofMonth  EndofQuarter  \
Stock Date                                                             
AAPL  2014-11-05  108.860001        4    2       False         False   
      2014-11-06  108.699997        4    3       False         False   
      2014-11-07  109.010002        4    4       False         False   
      2014-11-10  108.830002        4    0       False         False   
      2014-11-11  109.699997        4    1       False         False   
      2014-11-12  111.250000        4    2       False         False   
      2014-11-13  112.820000        4    3       False         False   
      2014-11-14  114.180000        4    4       False         False   
      2014-11-17  113.989998        4    0       False         False   
      2014-11-18  115.470001        4    1       False         False   

                  PreviousValue  Value2DaysAgo    Adj Close  
Stock Date                                                   
AAPL  2014-11-05            NaN            NaN  2023.569946  
      2014-11-06     108.860001            NaN  2031.209961  
      2014-11-07     108.699997     108.860001  2031.920044  
      2014-11-10     109.010002     108.699997  2038.260010  
      2014-11-11     108.830002     109.010002  2039.680054  
      2014-11-12     109.699997     108.830002  2038.250000  
      2014-11-13     111.250000     109.699997  2039.329956  
      2014-11-14     112.820000     111.250000  2039.819946  
      2014-11-17     114.180000     112.820000  2041.319946  
      2014-11-18     113.989998     114.180000  2051.800049

**Question 5:** Window Statistics

Lots of times, if we want to capture some idea of momentum, or how some value changes with what's usually observed.

Ie, if we had 48 purchases in a store today, how does that number compare to what's happened in the last 14 days?  Are things trending up or trending down?  

This also allows us to get a clearer picture of general trends in values, even if there are irregular daily spikes.

To handle these sorts of issues, pandas has an entire section to calculate window statistics called `rolling`, it works like this:

In [132]:
# I'll create a sample dataframe with 30 days worth of values
import numpy as np
index = pd.date_range(start='01/01/2020', end='02/05/2020')
sample_df = pd.DataFrame(np.random.randn(36), index=index, columns=['Value'])
# and here's what it looks like
sample_df.head()

,Value
2020-01-01,1.432834
2020-01-02,1.864196
2020-01-03,-0.079532
2020-01-04,-0.599474
2020-01-05,1.555828


In [133]:
# and now we'll see rolling 10 day averages
sample_df.rolling(10).mean()

,Value
2020-01-01,NaN
2020-01-02,NaN
2020-01-03,NaN
2020-01-04,NaN
2020-01-05,NaN
2020-01-06,NaN
2020-01-07,NaN
2020-01-08,NaN
2020-01-09,NaN
2020-01-10,0.144599


You can specify the number of observations to calculate, and choose your aggregator -- `mean()`, `min()`, `sum()`, etc, although `mean()` is the most common.

**Your Turn:** Calculate the rolling 5 & 10 day moving averages for each stock inside the dataset.

**Note:** Do *not* try and merge them back into your dataset yet, just make sure you have the values showing up.

In [135]:
df.groupby(level=0)['Price'].rolling(5).mean()

Stock  Stock  Date      
AAPL   AAPL   2014-11-05           NaN
              2014-11-06           NaN
              2014-11-07           NaN
              2014-11-10           NaN
              2014-11-11    109.020000
                               ...    
MSFT   MSFT   2019-10-28    139.694000
              2019-10-29    140.986001
              2019-10-30    142.460001
              2019-10-31    143.145999
              2019-11-01    143.744000
Name: Price, Length: 6285, dtype: float64

In [136]:
df.groupby(level=0)['Price'].rolling(10).mean()

Stock  Stock  Date      
AAPL   AAPL   2014-11-05           NaN
              2014-11-06           NaN
              2014-11-07           NaN
              2014-11-10           NaN
              2014-11-11           NaN
                               ...    
MSFT   MSFT   2019-10-28    139.598001
              2019-10-29    139.724000
              2019-10-30    140.144000
              2019-10-31    140.511999
              2019-11-01    141.142999
Name: Price, Length: 6285, dtype: float64

If you take a look at the index, you should notice that it has *three* levels to it, and not just two like before.  

Combining datasets with differing numbers of levels is cumbersome, and there's a decent amount of churn in what methods work from one version of Pandas to another.  

For now, try and get these values back into your original dataset by taking the following steps:

 - calculate the 5 & 10 rolling averages for each stock price on the multilevel index, and save these as variables, and then use the *values* attribute for each one to drop the index and just get the column values (ask me about this if you have questions)
 - use reset_index() to unstack the index on your original dataframe
 - create new columns for the 5 & 10 day moving averages in the original dataset, using the values from the first step.
 
So as a quick example, it would sort of work like this:

`five_day = df.groupby(level=0)['Price'].your_stuff_here.values`

And then use this as the basis to make your new column from your original dataframe with the reset index.

In [ ]:
# your answer here